In [1]:
pip install mysql-connector-python


Note: you may need to restart the kernel to use updated packages.


In [ ]:
import mysql.connector
from mysql.connector import errorcode

# Admin credentials (root or a user with privileges)
admin_user = "root"
admin_pass = "mayu"  # replace with your root password

# New chatbot user details
chatbot_user = "chatbot_user"
chatbot_pass = "mypassword"
chatbot_db = "chatbot_db"

try:
    # Connect as admin
    admin_conn = mysql.connector.connect(
        host="localhost",
        user=admin_user,
        password=admin_pass
    )
    admin_cursor = admin_conn.cursor()

    # 1️⃣ Create chatbot user
    admin_cursor.execute(f"CREATE USER IF NOT EXISTS '{chatbot_user}'@'localhost' IDENTIFIED BY '{chatbot_pass}'")

    # 2️⃣ Create database
    admin_cursor.execute(f"CREATE DATABASE IF NOT EXISTS {chatbot_db}")

    # 3️⃣ Grant privileges
    admin_cursor.execute(f"GRANT ALL PRIVILEGES ON {chatbot_db}.* TO '{chatbot_user}'@'localhost'")
    admin_cursor.execute("FLUSH PRIVILEGES")
    admin_conn.commit()

    print("Database and user setup completed!")

    # 4️⃣ Connect as chatbot_user and create reminders table
    chatbot_conn = mysql.connector.connect(
        host="localhost",
        user=chatbot_user,
        password=chatbot_pass,
        database=chatbot_db
    )
    chatbot_cursor = chatbot_conn.cursor()

    chatbot_cursor.execute("""
        CREATE TABLE IF NOT EXISTS reminders (
            id INT AUTO_INCREMENT PRIMARY KEY,
            text VARCHAR(255),
            created_at DATETIME
        )
    """)
    chatbot_conn.commit()
    chatbot_cursor.close()
    chatbot_conn.close()

    admin_cursor.close()
    admin_conn.close()

except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("❌ Something is wrong with your admin username or password")
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("❌ Database does not exist")
    else:
        print(err)

   


Database and user setup completed!


In [5]:
import mysql.connector
import random
import datetime
import re
import requests

class Chatbot:
    def __init__(self, name="PyBot"):
        self.name = name

        # --------------------------
        # Connect as chatbot_user
        # --------------------------
        self.conn = mysql.connector.connect(
            host="localhost",
            user="chatbot_user",    # MySQL username from setup
            password="mypassword",  # MySQL password from setup
            database="chatbot_db"
        )
        self.cursor = self.conn.cursor()

        # Keyword responses
        self.responses = {
            "hello": ["Hi there!", "Hello!", "Hey! How can I help you?"],
            "good morning": ["Good morning! Have a great day!", "Morning! How are you today?"],
            "good afternoon": ["Good afternoon! Hope you're doing well!"],
            "good evening": ["Good evening! How was your day?"],
            "bye": ["Goodbye! Have a nice day.", "See you later!"],
            "how are you": ["I'm just a bot, but I'm doing great!", "Doing well, thanks!"],
            "joke": ["Want a joke? Type 'joke'!"]
        }

    # ------------------------------
    # Reminders
    # ------------------------------
    def add_reminder(self, text):
        created_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        self.cursor.execute(
            "INSERT INTO reminders (text, created_at) VALUES (%s, %s)",
            (text, created_time)
        )
        self.conn.commit()
        return f"✅ Reminder added: {text}"

    def show_reminders(self):
        self.cursor.execute("SELECT id, text, created_at FROM reminders")
        rows = self.cursor.fetchall()
        if not rows:
            return "No reminders set."
        return "\n".join([f"{row[0]}. {row[1]} ({row[2]})" for row in rows])

    def delete_reminder(self, reminder_id):
        self.cursor.execute("DELETE FROM reminders WHERE id = %s", (reminder_id,))
        self.conn.commit()
        if self.cursor.rowcount:
            return f"✅ Deleted reminder ID: {reminder_id}"
        return "❌ Invalid reminder ID."

    # ------------------------------
    # Jokes
    # ------------------------------
    def get_joke(self):
        try:
            response = requests.get("https://official-joke-api.appspot.com/random_joke", timeout=5)
            data = response.json()
            return f"{data['setup']} ... {data['punchline']}"
        except:
            return "Sorry, I couldn't fetch a joke right now."

    # ------------------------------
    # Weather
    # ------------------------------
    def get_weather(self, city="London"):
        try:
            response = requests.get(f"https://wttr.in/{city}?format=3", timeout=5)
            return response.text
        except:
            return "Sorry, I couldn't fetch the weather right now."

    # ------------------------------
    # Respond to user input
    # ------------------------------
    def respond(self, user_input):
        user_input_lower = user_input.lower().strip()

        # 1️⃣ Reminder commands
        if user_input_lower.startswith("reminder:"):
            return self.add_reminder(user_input.split(":",1)[1].strip())
        elif user_input_lower == "show reminders":
            return self.show_reminders()
        elif user_input_lower.startswith("delete reminder"):
            parts = user_input_lower.split()
            if len(parts) == 3 and parts[2].isdigit():
                return self.delete_reminder(int(parts[2]))
            else:
                return "Specify a valid reminder ID. Example: delete reminder 1"

        # 2️⃣ Weather command
        if user_input_lower.startswith("weather"):
            parts = user_input_lower.split()
            city = parts[1] if len(parts) > 1 else "London"
            return self.get_weather(city)

        # 3️⃣ Keyword responses
        for keyword in self.responses:
            if re.search(rf"\b{keyword}\b", user_input_lower):
                reply = random.choice(self.responses[keyword])
                if keyword == "joke":
                    reply = self.get_joke()
                return reply

        # 4️⃣ Math evaluation
        if re.match(r"^[0-9+\-*/().\s]+$", user_input):
            try:
                return str(eval(user_input))
            except:
                return "I couldn't calculate that."

        # 5️⃣ Default fallback
        return ("Sorry, I didn’t understand that. You can say 'hello', 'good morning', "
                "'good evening', 'joke', 'reminder: <text>', 'show reminders', "
                "'delete reminder <id>', 'weather <city>', or type a math expression.")

    # ------------------------------
    # Close MySQL connection
    # ------------------------------
    def close(self):
        self.cursor.close()
        self.conn.close()


# ------------------------------
# Main Chat Loop
# ------------------------------
def main():
    bot = Chatbot()
    print(f"💬 Welcome to {bot.name}! Type 'bye' to exit.\n")

    while True:
        user_input = input("You: ").strip()
        if not user_input:
            continue

        reply = bot.respond(user_input)
        print(f"{bot.name}: {reply}\n")

        if "bye" in user_input.lower():
            break

    bot.close()

if __name__ == "__main__":
    main()



💬 Welcome to PyBot! Type 'bye' to exit.



You:  hello


PyBot: Hello!



You:  good morning


PyBot: Morning! How are you today?



...

PyBot: ✅ Reminder added: buy groceries



You:  show reminders


PyBot: 1. buy groceries (2025-09-16 12:57:08)



You:  bye


PyBot: See you later!

